In [28]:
import pandas as pd
import re
import nltk
import numpy as np

from nltk.corpus import sentiwordnet as swn

In [41]:
정상기업뉴스1 = pd.read_csv('정상기업번역파일.csv')
정상기업뉴스1.drop('뉴스기사본문전처리', axis=1, inplace=True)

정상기업뉴스2 = pd.read_csv('정상기업번역파일추가.csv')
정상기업뉴스2.drop('뉴스기사본문전처리', axis=1, inplace=True)

정상기업뉴스 = pd.concat([정상기업뉴스1, 정상기업뉴스2], axis=0)
정상기업뉴스.reset_index(inplace=True, drop=True)

정상기업뉴스['기사본문번역'] = [re.sub('[가-힣]', '', s) for s in 정상기업뉴스['기사본문번역']]

정상기업뉴스['기사발행일'] = 정상기업뉴스['기사발행일'].apply(lambda x: x[:4])
정상기업뉴스

,기업,기사발행일,기사제목,기사본문번역
0,씨제이이엔엠,2011,"씨제이이엔엠, 유형자산 취득 후 처분 예정",CJ ENM announced that it will acquire tangible...
1,씨제이이엔엠,2011,시초가 소폭 하락 예상 - 코스닥 동시호가,The Japanese KOSDAQ market is expected to star...
2,안랩,2011,"안랩, `V3ㆍ네트워크 보안장비 연동` 특허 획득",Ahn Cheol SooThe institute announced that the ...
3,안랩,2011,"한컴과 안랩, 10월은 특별한 달","In the wake of the month, Korea's leading soft..."
4,포스코아이씨티,2011,"포스코아이씨티, POSCO IT Outsourcing SLA 계약금액 증가",POSCO ICT announced that it would revise and d...
...,...,...,...,...
6873,에이치엘비,2020,"""이래도 불법 공매도가 아니라고?""...에이치엘비 주주들 뿔났다",HLB shareholders are raising suspicions that i...
6874,에이치엘비,2020,"리보세라닙 병용 식도암 임상, 방사선 병행시 3명중 1명 `완전관해`",HLB announced that it once again secured stati...
6875,에이치엘비,2020,"넥스트사이언스 관계사 나노젠 ""내년 5월 코로나백신 양산 계획""","Next Science announced that Nanogen, the only ..."
6876,에이치엘비,2020,넥스트사이언스 관계사 나노젠 “코로나 백신 내년 5월 양산 계획”,"Nanocobox, made from photoNextScience recombin..."


In [43]:
정상기업뉴스['기사본문번역토큰'] = 정상기업뉴스['기사본문번역'].apply(lambda x: nltk.word_tokenize(x))
정상기업뉴스['기사본문번역토큰'] = 정상기업뉴스['기사본문번역토큰'].apply(lambda x: nltk.pos_tag(x))

In [44]:
def word_sentiment_calculator(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set), neg_score/len(syn_set))

In [45]:
def sentence_sentiment_calculator(sent):
    tokens =  nltk.word_tokenize(sent)
    pos_tags = nltk.pos_tag(tokens)
    
    pos_score = 0
    neg_score = 0
    for word, tag in pos_tags:
        pos_score += word_sentiment_calculator(word, tag)[0]
        neg_score += word_sentiment_calculator(word, tag)[1]
    return (pos_score, neg_score)

In [46]:
## 긍정일경우 0 부정일경우 1
predicted = []

for i in range(len(정상기업뉴스)):
    scores = sentence_sentiment_calculator(정상기업뉴스.기사본문번역[i])
    
    if scores[0] >= scores[1]:
        predicted.append(0)
    else:
        predicted.append(1)

정상기업뉴스['긍정부정분류'] = predicted

In [67]:
a = pd.DataFrame(정상기업뉴스.groupby(['기업', '기사발행일']).count()['긍정부정분류'])
a.reset_index(drop = False, inplace = True)
a


,기업,기사발행일,긍정부정분류
0,경창산업,2013,62
1,고영테크놀러지,2011,47
2,광림,2016,55
3,금화피에스시,2013,71
4,나스미디어,2017,69
...,...,...,...
119,하이록코리아,2012,71
120,한국정보통신,2015,22
121,해성산업,2013,21
122,홈캐스트,2016,46


In [ ]:
a = pd.DataFrame(정상기업뉴스.groupby(['기업', '기사발행일']).count()['긍정부정분류'])
a.reset_index(drop = False, inplace = True)
a

기업 = 정상기업뉴스.groupby(['기업', '기사발행일']).count()['긍정부정분류'].index
뉴스 = 정상기업뉴스.groupby(['기업', '기사발행일']).count()['긍정부정분류'].values
전체기사수 = pd.DataFrame(columns=['기업명', '기사발행년도', '기사수'])
전체기사수['기업명'] = 기업
전체기사수['기사수'] = 뉴스

부정기사기업 = 정상기업뉴스[정상기업뉴스.긍정부정분류==1].groupby('기업', '기사발행일').count()['긍정부정분류'].index
부정기사뉴스 = 정상기업뉴스[정상기업뉴스.긍정부정분류==1].groupby('기업', '기사발행일').count()['긍정부정분류'].values
부정기사수 = pd.DataFrame(columns=['기업명', '기사발행년도', '부정기사수'])
부정기사수['기업명'] = 부정기사기업
부정기사수['부정기사수'] = 부정기사뉴스

정상기업 = pd.merge(전체기사수, 부정기사수, on=['기업', '기사발행일'], how='left')
정상기업['부정기사비율'] = 정상기업['부정기사수'] / 정상기업['기사수']
정상기업

In [74]:
전체기사수.columns = ['기업', '기사발행일', '긍정부정분류']

Index(['기업', '기사발행일', '긍정부정분류'], dtype='object')

In [118]:
전체기사수 = pd.DataFrame(정상기업뉴스.groupby(['기업', '기사발행일']).count()['긍정부정분류'])
전체기사수.reset_index(drop = False, inplace = True)
전체기사수.columns = ['기업', '기사발행일', '전체기사수']

부정기사수 = pd.DataFrame(정상기업뉴스[정상기업뉴스.긍정부정분류==1].groupby(['기업', '기사발행일']).count()['긍정부정분류'])
부정기사수.reset_index(drop = False, inplace = True)
부정기사수.columns = ['기업', '기사발행일', '부정기사수']

정상기업 = pd.merge(전체기사수, 부정기사수, on=['기업', '기사발행일'], how='left')
정상기업['부정기사비율'] = 정상기업['부정기사수'] / 정상기업['전체기사수']
정상기업.fillna(0, inplace=True)
정상기업['기사발행일'] = 정상기업['기사발행일'].astype(int)
정상기업['기사발행일'] = 정상기업['기사발행일'].apply(lambda x: x-2)
정상기업.drop(['전체기사수', '부정기사수'], axis=1, inplace=True)
정상기업.columns = ['회사명', '회계년도', '부정기사비율']

In [119]:
정상기업재무비율 = pd.read_csv('../재무비율모델/Dataset1_재무비율.csv')
정상기업재무비율 = 정상기업재무비율[정상기업재무비율.부실기업여부==0]
정상기업_감성분석 = pd.merge(정상기업재무비율, 정상기업, how='left', on=['회사명', '회계년도'])
정상기업_감성분석

,회사명,회계년도,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,폐지일자,부실기업여부,부정기사비율
0,씨제이이엔엠,2009,2.814262,16.478879,5.636979,174.850643,60.803587,2011-12-31,0,0.142857
1,안랩,2009,47.466492,50.400744,8.359978,545.217948,56.939152,2011-12-31,0,0.600000
2,포스코아이씨티,2009,-23.174026,-25.121698,6.080191,123.482176,130.063586,2011-12-31,0,0.185185
3,네오위즈,2009,46.036067,20.646980,19.274236,386.954876,93.022027,2011-12-31,0,0.250000
4,젬백스&카엘,2009,23.314514,-0.883477,-8.765436,238.951956,12.507727,2011-12-31,0,0.121495
...,...,...,...,...,...,...,...,...,...,...
130,지니뮤직,2016,45.063046,-14.623948,4.450491,262.390477,101.096537,2018-12-31,0,0.047619
131,대주전자재료,2016,-8.786573,9.737698,2.277756,170.648920,48.084200,2018-12-31,0,0.109589
132,크리스에프앤씨,2016,30.558431,46.651963,19.919396,213.287219,133.252743,2018-12-31,0,0.052632
133,앤디포스,2016,81.281282,35.275974,18.934118,1071.393920,79.493948,2018-12-31,0,0.087719


In [122]:
정상기업_감성분석[정상기업_감성분석.부정기사비율.isnull()]

,회사명,회계년도,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,폐지일자,부실기업여부,부정기사비율
12,엘엑스세미콘,2009,71.707134,71.106213,31.029620,487.021421,162.322089,2011-12-31,0,NaN
18,그래디언트,2009,54.232350,57.817313,-3.810019,388.233595,62.624130,2011-12-31,0,NaN
21,삼표시멘트,2009,-24.534618,-0.454552,-0.231312,166.851351,33.586567,2011-12-31,0,NaN
25,오성첨단소재,2009,-27.890561,10.049752,2.475594,132.422676,26.083271,2011-12-31,0,NaN
46,슈프리마에이치큐,2010,68.362503,31.135026,8.201998,1456.194901,33.145254,2012-12-31,0,NaN
55,모다이노칩,2010,44.058963,59.257566,10.137530,431.574725,70.480043,2012-12-31,0,NaN
56,동성화인텍,2010,-11.230463,-4.502102,4.940514,147.280635,78.125575,2012-12-31,0,NaN
66,엔에이치엔벅스,2010,45.697937,24.510821,7.787333,381.579100,61.721375,2012-12-31,0,NaN
72,슈프리마에이치큐,2011,68.193263,34.538444,5.174088,1251.529450,39.899231,2013-12-31,0,NaN
98,현대바이오랜드,2013,20.565094,56.405047,9.429258,405.724953,49.834653,2015-12-31,0,NaN
